In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model
import docplex.mp.solution as Solucion
import queue  
from collections import namedtuple
from dijkstar import Graph, find_path
import random


## get all edges passed by node

In [2]:
def get_edges (node) :
    eds = []
    for n in edges :
        if (n[0] == node):
            eds.append(n)
    return eds        

## return all pathes between 2 nodes

In [3]:
def total_costs (start , end  , nodes_passed  ):
    path = []
    cost = 0
    ##copy list by value no reference for the nodes that i have already passed
    my_nodes = nodes_passed.copy()
    ###check if by a node have already passed
    bol = start in nodes_passed
    if(bol == True) :
            return False ,[]
    elif start == end :
            return (True , [tuple([end,end])])
    my_nodes.append(start)
    ###  
    reached = False
    for n in get_edges(start) :
        ## lock for path for every edge that start have
        find , reached_pathed = total_costs(n[1],end,my_nodes)
        ### check if by this edge have found path to end and if yes save the path + this edge 
        if find == True:
            reached = True
            lists = False
            ##add the edge to all pathes found
            for o in reached_pathed :
                if type(o) == list:
                    lists = True 
            #check if reached_pathed has more than on path
            if lists:
                for p in range (len (reached_pathed)):
                    reached_pathed[p].append(n)
            else :    
                reached_pathed.append(n)      
            ### if the path is empty and reached_pathed[0] is of type list then path = reached_pathed
            if (len(path) == 0 and type (reached_pathed[0]) == list):
                path = reached_pathed
            else :    
                #check if reached_pathed is list of lists
                if (type (reached_pathed[0]) == list):
                    for p in reached_pathed:
                         path.append(p)
                else:        
                    path.append(reached_pathed)    
    return reached ,path

## return the cost of the backup path passed in pathes

In [4]:
def cost_backup_path (pathes ,dis):
    path_num = len(pathes)
    cost_total = 0
    for path in pathes:
        #print (path)
        for p in range(1,len(path)):
          #print(path[p])  
          cost_total += dis[path[p]]
    return cost_total/path_num

## return the cost of the pathes in pathes

In [5]:
def get_pathes_costs (pathes ,dis):
    paths_cost = []
    for path in pathes:
        cost = 0
        for p in range(1,len(path)):  
          cost += dis[path[p]]
        paths_cost.append(cost)
    return paths_cost

## create  graph 

In [7]:


l =[0,1,2,3,4]
edges = [(i,i+1) for i in l]
nodes = list(range(0,6))
edges1 = [(i,1+i) for i in l]
for ed in edges1:
    edges.append(tuple([ed[1],ed[0]]))
edges.append(tuple([0,3]))
edges.append(tuple([5,0]))
edges.append(tuple([0,5]))
edges.append(tuple([3,0]))

distences = {i: 1 for i in edges}
print (nodes)
print  (distences)

[0, 1, 2, 3, 4, 5]
{(0, 1): 1, (1, 2): 1, (2, 3): 1, (3, 4): 1, (4, 5): 1, (1, 0): 1, (2, 1): 1, (3, 2): 1, (4, 3): 1, (5, 4): 1, (0, 3): 1, (5, 0): 1, (0, 5): 1, (3, 0): 1}


## creating  graph using dijkstar class 

In [8]:
starter = nodes[0]
end = nodes[-1]
node_pathes = dict()
nodes_passed = []
graph = Graph()
cost_func = lambda u, v, e, prev_e: e['cost']
for ed in distences:
    graph.add_edge(ed[0], ed[1], {'cost':distences[ed] })  


## get the cost of backup path and primery path of all nodes using dijkstar class

In [9]:
backup_costs = {}
primery_costs = {}
for i in nodes:
    for i1 in nodes:
        if (i1 != i):
            primery_costs[(i,i1)] = list(find_path(graph,i , i1, cost_func=cost_func))[-1]
            total = total_costs (i , i1  , nodes_passed)
            backup_costs[(i,i1)] = cost_backup_path (total[1],distences)

## get the cost of backup path and primery path of all nodes without using dijkstar class (another manner)

In [10]:
backup_costs = {}
primery_costs = {}
for i in nodes:
    for i1 in nodes:
        if (i1 != i):
            total = total_costs (i , i1  , nodes_passed)
            pathes = get_pathes_costs (total[1],distences)
            primery_costs[(i,i1)] = min (pathes)
            backup_costs[(i,i1)] = sum(pathes)/len(pathes)

In [11]:
for i,i1 in zip (nodes,nodes):
    primery_costs[(i,i1)]=0
    backup_costs[(i,i1)]=0

## initialize variables 

In [14]:
controllers = nodes
switches = nodes
n_controllers = 3
alpha1 = 0.3
alpha2 = 0.7

## Create cplex model

In [15]:
mdl = Model("larc")

In [16]:
alledges=[(i,i1) for i in nodes for i1 in nodes ]

In [17]:
switch=mdl.binary_var_dict(nodes,name='switch')
cont=mdl.binary_var_dict(nodes,name='cont')
cost =mdl.continuous_var_dict(nodes,name='cost')
yik =mdl.binary_var_dict(zip(nodes,nodes),name='yik')
xik = mdl.binary_var_dict(alledges ,name='xik')

## define objective function 

In [18]:
mdl.minimize(mdl.sum((cont[con]*backup_costs[(con1,sw)]*alpha2*xik[sw,con1]) + (xik[sw,con1]*alpha1*cont[con]*primery_costs[(con1,sw)]) for con in nodes for sw in nodes for con1 in nodes))

## define  constraints

In [20]:
mdl.add_constraint(mdl.sum(cont[i] for i in nodes)==n_controllers )
for i in nodes :
    mdl.add_constraint(mdl.sum(xik[i,j] for j in nodes) == 1)
mdl.add_indicator_constraints(mdl.add_indicator(cont[j],xik[i,j] == 0 , 0) for j in nodes for i in nodes ) 

[docplex.mp.constr.IndicatorConstraint[](cont_0,xik_0_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_0,xik_1_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_0,xik_2_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_0,xik_3_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_0,xik_4_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_0,xik_5_0 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_0_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_1_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_2_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_3_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_4_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_1,xik_5_1 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_2,xik_0_2 == 0,true=0),
 docplex.mp.constr.IndicatorConstraint[](cont_2,xik_1_2 == 0,true=0),
 docplex.mp.constr.I

## solve

In [21]:
solucion = mdl.solve(log_output=True)

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 36 rows and 18 columns.
MIP Presolve added 360 rows and 180 columns.
Reduced MIP has 403 rows, 222 columns, and 834 nonzeros.
Reduced MIP has 222 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.18 ticks)
Found incumbent of value 41.000000 after 0.00 sec. (0.32 ticks)
Probing time = 0.00 sec. (0.17 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 180 rows and 0 columns.
MIP Presolve modified 30 coefficients.
Aggregator did 30 substitutions.
Reduced MIP has 193 rows, 192 columns, and 564 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.89 ticks)
Probing time = 0.00 sec. (0.13 ticks)
Tried aggregator 1 time.
Reduced MIP has 193 rows, 192 columns, and 564 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.65 ticks)
Probing time = 0.00 sec. (0.

In [22]:
solucion.display()

solution for: larc
objective: 20.200
cont_1 = 1
cont_3 = 1
cont_4 = 1
xik_0_3 = 1
xik_1_1 = 1
xik_2_3 = 1
xik_3_3 = 1
xik_4_4 = 1
xik_5_4 = 1


In [23]:
print(mdl.export_to_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: larc

Minimize
 obj: [ 4.800000000000 cont_0*xik_0_1 + 4.933333333333 cont_0*xik_0_2
      + 3.866666666667 cont_0*xik_0_3 + 4.933333333333 cont_0*xik_0_4
      + 4.800000000000 cont_0*xik_0_5 + 4.800000000000 cont_0*xik_1_0
      + 4.800000000000 cont_0*xik_1_2 + 4.933333333333 cont_0*xik_1_3
      + 6.700000000000 cont_0*xik_1_4 + 6.100000000000 cont_0*xik_1_5
      + 4.933333333333 cont_0*xik_2_0 + 4.800000000000 cont_0*xik_2_1
      + 4.800000000000 cont_0*xik_2_3 + 6.100000000000 cont_0*xik_2_4
      + 6.700000000000 cont_0*xik_2_5 + 3.866666666667 cont_0*xik_3_0
      + 4.933333333333 cont_0*xik_3_1 + 4.800000000000 cont_0*xik_3_2
      + 4.800000000000 cont_0*xik_3_4 + 4.933333333333 cont_0*xik_3_5
      + 4.933333333333 cont_0*xik_4_0 + 6.700000000000 cont_0*xik_4_1
      + 6.100000000000 cont_0*xik_4_2 + 4.800000000000 cont_0*xik_4_3
      + 4.800000000000 cont_0*xik_4_5 + 4.800000000000 cont_0*xik_